# Image Preprocessing

In [23]:
# Importing Libraries

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6b6e912f7eac460e813c136094c064e3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='5iM4qWKjVM6YNNr3cFCfcTjBx8l8fE45F8bOdJndp8Y7',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_6b6e912f7eac460e813c136094c064e3.get_object(Bucket='aibasedrealtimecommunication-donotdelete-pr-uyelwdfsxweesp', Key='Dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [25]:
pwd

'/home/wsuser/work'

In [26]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)


In [27]:
# Image Augmentation

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [28]:
# Loading train and test set

X_train = train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/training_set", target_size = (64, 64), batch_size = 32, class_mode = 'categorical')
X_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/test_set", target_size = (64, 64), batch_size = 32, class_mode = 'categorical')

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [29]:
# checking indices

X_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

# Model Building

In [30]:
# Importing Libraries

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten

In [31]:
# Initializing the Model

model = Sequential()

In [32]:
# Adding Convolution Layer

model.add(Convolution2D((32), (3,3), input_shape = (64, 64, 3), activation = 'relu'))

In [33]:
# Adding Pooling Layer

model.add(MaxPooling2D(pool_size = (2, 2)))

In [34]:
# Adding Flatten Layer

model.add(Flatten())

In [35]:
# Adding Hidden Layer

model.add(Dense(units = 512, kernel_initializer = 'random_uniform', activation = 'relu'))

In [36]:
# Adding Output Layer

model.add(Dense(units = 9, kernel_initializer = 'random_uniform', activation = 'softmax'))

In [37]:
# Compile the model

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [38]:
# Fiiting the model

model.fit_generator(X_train, steps_per_epoch = 24, epochs = 10, validation_data = X_test, validation_steps = 40)

/tmp/wsuser/ipykernel_236/1270027362.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(X_train, steps_per_epoch = 24, epochs = 10, validation_data = X_test, validation_steps = 40)


Epoch 1/10
24/24 [==============================] - 7s 282ms/step - loss: 1.2710 - accuracy: 0.5703 - val_loss: 0.6611 - val_accuracy: 0.7641
Epoch 2/10
24/24 [==============================] - 6s 251ms/step - loss: 0.4401 - accuracy: 0.8438 - val_loss: 0.4736 - val_accuracy: 0.8648
Epoch 3/10
24/24 [==============================] - 6s 272ms/step - loss: 0.2849 - accuracy: 0.9219 - val_loss: 0.3455 - val_accuracy: 0.9195
Epoch 4/10
24/24 [==============================] - 7s 275ms/step - loss: 0.1931 - accuracy: 0.9414 - val_loss: 0.3100 - val_accuracy: 0.9164
Epoch 5/10
24/24 [==============================] - 7s 274ms/step - loss: 0.1410 - accuracy: 0.9570 - val_loss: 0.2939 - val_accuracy: 0.9281
Epoch 6/10
24/24 [==============================] - 7s 277ms/step - loss: 0.1432 - accuracy: 0.9622 - val_loss: 0.2978 - val_accuracy: 0.9438
Epoch 7/10
24/24 [==============================] - 6s 264ms/step - loss: 0.1128 - accuracy: 0.9648 - val_loss: 0.2513 - val_accuracy: 0.9414
Epoch 

In [39]:
# Saving the model

model.save('aslpng1.h5')

In [52]:
!tar -zcvf ai-based-real-time-classification-model.tgz aslpng1.h5

aslpng1.h5


In [41]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 23.2 MB/s eta 0:00:01


In [42]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey" : "T7rpH1KfTn-s-zfDCmTyeArxYrcvFGHFV21qTDW5pf5x"
}
client = APIClient(wml_credentials)

In [43]:
def guid_space_name(client, ai_based_real_time_communication_deploy_space):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == ai_based_real_time_communication_deploy_space)['metadata']['id'])

In [45]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/5be23fa7fba94c8aa2e3db0b2a4db8d2:04f159f4-9ffb-4e0d-b70b-2a1f3b216970::',
      'guid': '04f159f4-9ffb-4e0d-b70b-2a1f3b216970',
      'name': 'Watson Machine Learning-av',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'ai_based_real_time_communication_deploy_space',
    'scope': {'bss_account_id': '5be23fa7fba94c8aa2e3db0b2a4db8d2'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '0a42d73b-35af-4f9d-92da-4a84147fcb1c',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': 'cf70aa9a76bc4588bfcf319cfbcf967a',
        'api_key': 'UH0R4o_Q7uIIrJQDiKxtgUw1thaMQfNcxnzibJ3mmjn0',
        'secret_access_key': '2ab989f675cc399997783bb5bc124440ac7da6f8c59d68a2',
        'service_id': 'ServiceId-ca6e7997-e0a7-43a6-95ed-c5129491ab68'},
       'editor': {'access_key_id': '6639c313223b4eccaf

In [47]:
space_id = guid_space_name(client, 'ai_based_real_time_communication_deploy_space')
space_id

'1853d74e-ca3c-4075-81e3-d5cdd0741a52'

In [48]:
client.set.default_space(space_id)

'SUCCESS'

In [50]:
client.software_specifications.list(100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [51]:
software_space_id = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_id

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [53]:
model_details = client.repository.store_model(model = 'ai-based-real-time-classification-model.tgz', meta_props = {
    client.repository.ModelMetaNames.NAME:"CNN Model Buiding",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_id
})

In [54]:
model_id = client.repository.get_model_id(model_details)
model_id

'59b18265-3a03-47d3-b2d8-d9a0c5106f05'

In [56]:
client.repository.download(model_id, 'ai-based-real-time-classification-model.h5')

Successfully saved model content to file: 'ai-based-real-time-classification-model.h5'


'/home/wsuser/work/ai-based-real-time-classification-model.h5'